<a href="https://colab.research.google.com/github/kianpu34593/NotSoFastMD/blob/main/src/Base_Code__for_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from random import sample
import numba
import time
from tqdm.notebook import tqdm
plt.rcParams['figure.figsize'] = [8.0, 6.0]
#if using google colab
from google.colab import files
uploaded = files.upload()

Saving liquid256.txt to liquid256.txt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# %%
#Randomlly Initialize the Velocities
@numba.njit
def random_vel_generator(n=256,T_equal=100,m=1,e_scale=1): # n=number of particles
    total_k=3*T_equal*(n-1)/2 #dimensionless
    vel_per_particle=np.zeros((n,3))
    for axis in range(3):
        vel_per_particle[:,axis]=np.random.randn(256,)-0.5
    Mom_x_total=np.sum(vel_per_particle[:,0])
    Mom_y_total=np.sum(vel_per_particle[:,1])
    Mom_z_total=np.sum(vel_per_particle[:,2])
    Mom_x_avg=Mom_x_total/n
    Mom_y_avg=Mom_y_total/n
    Mom_z_avg=Mom_z_total/n
    vel_per_particle=vel_per_particle-np.array([Mom_x_avg,Mom_y_avg,Mom_z_avg]).reshape(1,3)
    k_avg_init=0.5*(1/n)*np.sum(np.sum(vel_per_particle**2,axis=1),axis=0)
    k_avg_T_eq=total_k/n
    scaling_ratio=np.sqrt(k_avg_T_eq/k_avg_init)
    vel_per_particle=vel_per_particle*scaling_ratio
    return vel_per_particle


In [ ]:
#function of pbc rule2
@numba.njit
def pbc2(separation=0,L=6.8):
    separation_new=np.zeros((separation.shape[0],3))   
    for i in range(separation.shape[0]):
        separation_ind=separation[i,:]
        separation_empty=np.zeros((1,3))
        for j in range(3):
            separation_axis=numba.float64(separation_ind[j])
            
            if separation_axis < -L/2:
                separation_axis_new=separation_axis+L
            elif separation_axis > L/2:
                separation_axis_new=separation_axis-L
            else:
                separation_axis_new=separation_axis
            separation_empty[:,j]=separation_axis_new
        separation_new[i,:]=separation_empty
    return separation_new

In [ ]:
#function of pbc rule1
@numba.njit
def pbc1(position=0,L=6.8,counter=None):
    x_new=np.empty((position.shape[0],3))   
    for i in range(position.shape[0]):
        position_ind=position[i,:]
        position_empty=np.empty((1,3))
        for j in range(3):
            position_axis=numba.float64(position_ind[j])
            if position_axis < 0:
                position_axis_new=position_axis+L
                if counter!=None:
                    counter[i,j]+=-1
            elif position_axis > L:
                position_axis_new=position_axis-L
                if counter!=None:
                    counter[i,j]+=1
            else:
                position_axis_new=position_axis
            position_empty[:,j]=position_axis_new
        x_new[i,:]=position_empty
    return x_new, counter

In [ ]:
# %%
#function of dimensionless LJ: U(r)=4(r^{-12}-r^{-6})
#define the function of acceleration of LJ potential with cut_off
@numba.njit
def LJ_accel(position=1,r_cut=2.5,L=6.8):
    num=position.shape[0]
    update_accel=np.zeros((num,3))
    dU_drcut=48*r_cut**(-13)-24*r_cut**(-7) #evaluate at r_cutoff
    for atom in range(num):
        position_other=np.concatenate((position[0:atom,:],position[atom+1:,:]),axis=0)
        position_atom=position[atom]
        separation=position_atom-position_other   
        separation_new=pbc2(separation=separation,L=L)
        r_relat=np.sqrt(np.sum(separation_new**2,axis=1))
        accel=np.zeros((r_relat.shape[0],3))
        #get out the particles inside the r_limit
        for i, r0 in enumerate(r_relat):
            if r0 <= r_cut:
               separation_active_num=separation_new[i,:]
               vector_part=separation_active_num*(1/r0)
               scalar_part=48*r0**(-13)-24*r0**(-7)-dU_drcut
               accel_num=vector_part*scalar_part
               accel[i,:]=accel_num
        update_accel[atom,:]=np.sum(accel,axis=0)
        
    return update_accel.reshape(num,3)

In [ ]:
# %%
#Update rule: velocity Verlet
@numba.njit
def vel_Ver(dt=0,x_0=0,x_dot_0=0,r_limit=2.5,L=6.8, counter=np.zeros((1,3))):
    #pbc rule 1
    x_0,counter=pbc1(position=x_0,L=L,counter=counter)
    
    #acceleration at 0
    accel_0=LJ_accel(position=x_0,r_cut=r_limit,L=L)
    
    #velocity at dt/2
    x_dot_interm=x_dot_0+accel_0*(dt/2)
    
    #position at dt
    x_final=x_0+x_dot_interm*dt
    
    #PBC rule 1
    x_final,counter=pbc1(position=x_final,L=L,counter=counter)
        
    #acceleration at dt
    accel_final=LJ_accel(position=x_final,r_cut=r_limit,L=L)
    
    #velocity at dt
    x_dot_final=x_dot_interm+accel_final*(dt/2)
    
    #collect info
    info=np.concatenate((x_final,x_dot_final,accel_final),axis=1)
    return info,counter #Data shape(N,9) [position_(xyz),velocity_(xyz),force(accel if dimension)_xyz]

In [ ]:
# %%
#Update rule: velocity Verlet with Thermo Stats
#@numba.njit
def vel_Ver_ts(dt=0,x_0=0,x_dot_0=0,r_limit=2.5,L=6.8,t_damp=0.05,tau_0=0,T_insta=120, T_des=100):
    #pbc rule 1
    x_0,counter=pbc1(position=x_0,L=L)
    
    #acceleration at 0
    accel_0=LJ_accel(position=x_0,r_cut=r_limit,L=L)

    #velocity at dt/2
    x_dot_interm=x_dot_0+(accel_0-tau_0*x_dot_0)*(dt/2)
    
    #position at dt
    x_final=x_0+x_dot_interm*dt
    
    #update tau
    tau_final = tau_0 + (1/t_damp**2)*(T_insta/T_des - 1)*dt
    
    #PBC rule 1
    x_final,counter=pbc1(position=x_final,L=L)
        
    #acceleration at dt
    accel_final=LJ_accel(position=x_final,r_cut=r_limit,L=L)
    
    #velocity at dt
    x_dot_final=(x_dot_interm+accel_final*(dt/2))/(1+tau_final*(dt/2))
    
    #collect info
    info=np.concatenate((x_final,x_dot_final,accel_final),axis=1)
    return tau_final, info #Data shape(N,9) [position_(xyz),velocity_(xyz),force(accel if dimension)_xyz]

In [ ]:
# %%
#Function for avg KE (the system) 
@numba.njit
def Kin_Eng(m=1,x_dot=1):
    num=x_dot.shape[0]
    Kinetic_per=0.5*m*np.sum(x_dot**2,axis=1)
    Kinetic_sum=np.sum(Kinetic_per,axis=0)
    return Kinetic_sum

In [ ]:
# %%
#Function for avg LJ potential with cut_off
@numba.njit
def LJ_potent_nondimen(position=1,r_cut=2.5,L=6.8):
    num=position.shape[0]
    update_LJ=np.zeros((num-1,1))
    
    #fix value for a certain r_limit
    dU_drcut=24*r_cut**(-7)-48*r_cut**(-13)
    U_rcut=4*(r_cut**(-12)-r_cut**(-6))
    for ind, atom in enumerate(range(num-1)):
        position_relevent=position[atom:,:]
        position_other=position_relevent[1:,:]
        
        #pbc rule2
        separation=position_relevent[0,:]-position_other
        separation_new=pbc2(separation=separation,L=L)
        
        r_relat=np.sqrt(np.sum(separation_new**2,axis=1)).reshape(separation_new.shape[0],)
        LJ=[]
        #get out the particles inside the r_limit
        for i, r0 in enumerate(r_relat):
            if r0 <= r_cut:
               LJ_num=4*r0**(-12)-4*r0**(-6)-U_rcut-(r0-r_cut)*dU_drcut
               LJ.append(LJ_num)
            update_LJ[atom,:]=np.sum(np.array(LJ),axis=0)    
    return np.sum(update_LJ)

In [ ]:
# %%
#Function of insta Pressue
@numba.njit
def insta_pressure(L=6.8,T=10,N=256,position=1,r_cut=2.5,e_scale=1):
    num=position.shape[0]
    k_B=1.38064852*10**(-23)
    energy_scale = (k_B*T/e_scale)
    V=L**3
    pres_ideal=N*(1/V)*energy_scale
    dU_drcut=24*r_cut**(-7)-48*r_cut**(-13)
    pres_virial=np.zeros((num-1,1))
    for atom in range(num-1):
        position_relevent=position[atom:,:]
        position_other=position_relevent[1:,:]
        position_atom=position_relevent[0,:]
        
        #pbc rule 2
        separation=position_atom-position_other
        separation_new=pbc2(separation=separation,L=L)
        
        r_relat=np.sqrt(np.sum(separation_new**2,axis=1)).reshape(separation_new.shape[0],)
        force=[]
        active_r_relat=[]
        #get out the particles inside the r_limit
        for i, r0 in enumerate(r_relat):
            if r0 <= r_cut:
               #active_r_relat.append(r0)
               force_num=-(24*r0**(-7)-48*r0**(-13))+dU_drcut
               force.append(force_num)
               active_r_relat.append(r0)     
        ##get out the particles inside the r_cut
        active_amount=np.array(active_r_relat).shape[0]
        rijFij=np.array(active_r_relat).reshape(1,active_amount)@np.array(force).reshape(active_amount,1)
        pres_virial[atom,:]=rijFij
    pres_insta=pres_ideal+np.sum(pres_virial,axis=0)/(3*V) #Double Count?
    return pres_insta

In [ ]:
# %%
#The LJ potential MD simulation function
#@numba.njit
def LJ_MD(position_init=0,dt=0.002,stop_step=1000,accel_init=0,r_cut=2.5,L=6.8,T_eq=100,e_scale=1,sig=1,tau=0, t_damp=0.05,thermal_stat=True,prior_Vel=None,prior_accel=None):
    k_B=1.38064852*10**(-23)
    #NVT or NVE
    if thermal_stat==True:
        #import position
        #position_init=np.loadtxt(file_name)
        size_sim=position_init.shape[0]
    
        #velocity initialization 
        x_dot_init=random_vel_generator(n=size_sim,T_equal=T_eq,m=1,e_scale=e_scale)       
        accel_init=np.zeros((size_sim,3))
        
        #initialize the info matrix
        info=np.empty((stop_step+1,size_sim,9))
        info[0,:,:]=np.concatenate((position_init,x_dot_init,accel_init),axis=1)

        #initialize PE, KE, T_insta, P_insta, Momentum
        PE=np.empty((stop_step+1,1))
        KE=np.empty((stop_step+1,1))
        T_insta=np.empty((stop_step+1,1))
        P_insta=np.empty((stop_step+1,1))

        for step in tqdm(range(stop_step)):
            #update and store PE,KE,T_insta,P_insta from 0 to step-1
            PE[step,:]=LJ_potent_nondimen(info[step,:,0:3],r_cut=r_cut,L=L) #dimensionless
            KE[step,:]=Kin_Eng(m=1,x_dot=info[step,:,3:6]) #dimensionless
            T_insta[step,:]=2*KE[step,:]*e_scale/(3*(size_sim-1)*k_B) #k
            P_insta[step,:]=insta_pressure(L=L,T=T_insta[step,:],N=size_sim,position=info[step,:,0:3],r_cut=r_cut,e_scale=e_scale)*(e_scale/(L*sig*(10**(-10)))**3) #Pa
            T_insta_feed=T_insta[step,:]*(k_B/e_scale)

            #update position, velocity and acceleratio with thermal stats
            tau, info_temp=vel_Ver_ts(dt=dt, x_0=info[step,:,0:3], x_dot_0=info[step,:,3:6], r_limit=r_cut, L=L, t_damp=t_damp, tau_0=tau,T_insta=T_insta_feed, T_des=T_eq)
            info[step+1,:,:]=info_temp.reshape(1,size_sim,9)
        PE[-1,:]=LJ_potent_nondimen(info[-1,:,0:3],r_cut=r_cut,L=L) #dimensionless
        KE[-1,:]=Kin_Eng(m=1,x_dot=info[-1,:,3:6]) #dimensionless
        T_insta[-1,:]=2*KE[-1,:]*e_scale/(3*(size_sim-1)*k_B) #k
        P_insta[-1,:]=insta_pressure(L=L,T=T_insta[-1,:],N=size_sim,position=info[-1,:,0:3],r_cut=r_cut,e_scale=e_scale)*(e_scale/(L*sig*(10**(-10)))**3) #Pa
        msd_mat=None
    elif thermal_stat==False:
        #import position
        #position_init=np.loadtxt(file_name)
        size_sim=position_init.shape[0]
    
        #velocity and acceleration initialization 
        x_dot_init=prior_Vel.reshape(size_sim,3)       
        accel_init=prior_accel.reshape(size_sim,3)

        #initialize the info matrix and mean_sqaure_error mat
        info=np.empty((stop_step+1,size_sim,9))
        info[0,:,:]=np.concatenate((position_init,x_dot_init,accel_init),axis=1)
        msd_mat=np.empty((stop_step+1,1))
        msd_counter=np.zeros((size_sim,3))
        cell=np.ones((size_sim,3))*L

        #save the origin
        origin=info[0,:,0:3]

        #initialize PE, KE, T_insta, P_insta, Momentum
        PE=np.empty((stop_step+1,1))
        KE=np.empty((stop_step+1,1))
        T_insta=np.empty((stop_step+1,1))
        P_insta=np.empty((stop_step+1,1))

        for step in tqdm(range(stop_step)):
            #update and store PE,KE,T_insta,P_insta from 0 to step-1
            PE[step,:]=LJ_potent_nondimen(info[step,:,0:3],r_cut=r_cut,L=L) #dimensionless
            KE[step,:]=Kin_Eng(m=1,x_dot=info[step,:,3:6]) #dimensionless
            T_insta[step,:]=2*KE[step,:]*e_scale/(3*(size_sim-1)*k_B) #k
            P_insta[step,:]=insta_pressure(L=L,T=T_insta[step,:],N=size_sim,position=info[step,:,0:3],r_cut=r_cut,e_scale=e_scale)*(e_scale/(L*sig*(10**(-10)))**3) #Pa

            #update Mean-sqaure-displacement
            disp=info[step,:,0:3]+np.multiply(msd_counter,cell)
            msd_mat[step,:]=np.mean(np.sum((disp-origin)**2,axis=1))

            #update position, velocity and acceleratio with thermal stats
            info_temp,msd_counter=vel_Ver(dt=dt, x_0=info[step,:,0:3], x_dot_0=info[step,:,3:6], r_limit=r_cut, L=L, counter=msd_counter)
            info[step+1,:,:]=info_temp.reshape(1,size_sim,9)

        PE[-1,:]=LJ_potent_nondimen(info[-1,:,0:3],r_cut=r_cut,L=L) #dimensionless
        KE[-1,:]=Kin_Eng(m=1,x_dot=info[-1,:,3:6]) #dimensionless
        T_insta[-1,:]=2*KE[-1,:]*e_scale/(3*(size_sim-1)*k_B) #k
        P_insta[-1,:]=insta_pressure(L=L,T=T_insta[-1,:],N=size_sim,position=info[-1,:,0:3],r_cut=r_cut,e_scale=e_scale)*(e_scale/(L*sig*(10**(-10)))**3) #Pa
        disp=info[-1,:,0:3]+np.multiply(msd_counter,cell)
        msd_mat[-1,:]=np.mean(np.sum((disp-origin)**2,axis=1))
    return info,PE,KE,T_insta,P_insta,msd_mat

In [ ]:
#hyper parameter
T_dimensional_equal_ls=np.linspace(210,290,5)
thermal_stat_ls=[True,False]
dt=0.002
stop_step=int(60/dt)
k_B=1.38064852*10**(-23)
r_c=2.5
L0=6.8
energy_scale=1.66*10**(-21)#unit: J, Argon
sigma=3.4 #unit: Ang, Argon 
file_name_on='liquid256.txt' #if thermal stat is on
tau=0
t_damp=0.05
for T_dimensional_equal in T_dimensional_equal_ls:
    file_name_off='/content/drive/My Drive/PS4_EC/NVT_pos_vel_acc_last_T={}K.txt'.format(T_dimensional_equal) #if thermal stat is off
    for thermal_stat in thermal_stat_ls:
        start_time=time.time()
        if thermal_stat==True:
          position_init=np.loadtxt(file_name_on)
          a_init=np.zeros((256,3))
          T_equal=T_dimensional_equal*k_B/energy_scale
          info,PE,KE,T_insta,P_insta,msd_mat=LJ_MD(position_init=position_init,
                                            dt=dt,
                                            stop_step=stop_step,
                                            r_cut=r_c,
                                            L=L0,
                                            T_eq=T_equal,
                                            e_scale=energy_scale,
                                            sig=sigma,
                                            tau=tau,
                                            t_damp=t_damp,
                                            thermal_stat=thermal_stat,
                                            prior_Vel=None,
                                            prior_accel=None)
          saving_NVT(info,T_dimensional_equal,T_insta,P_insta)
        elif thermal_stat==False:
            general=np.loadtxt(file_name_off,skiprows=1)
            position_init = general[:,0:3]
            vel_init = general[:,3:6]
            accel_init = general[:,6:]
            info_temp,PE_temp,KE_temp,T_insta_temp,P_insta_temp,msd_mat=LJ_MD(position_init=position_init,
                                                                              dt=dt,
                                                                              stop_step=stop_step,
                                                                              r_cut=r_c,
                                                                              L=L0,
                                                                              T_eq=None,
                                                                              e_scale=energy_scale,
                                                                              sig=sigma,
                                                                              tau=None,
                                                                              t_damp=None,
                                                                              thermal_stat=thermal_stat,
                                                                              prior_Vel=vel_init,
                                                                              prior_accel=accel_init)
            saving_NVE(T_insta_temp,P_insta_temp,KE_temp,PE_temp,info_temp)     
        end_time=time.time()
        print("T={}, with thermal_stat={}, it took:".format(T_dimensional_equal,thermal_stat),np.round(end_time-start_time,decimals=1),'s')